In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import pickle
import numpy as np
f1 = open('data/split_0.80_0.10_0.10_aug_to_2500_down_sample_True_instance0-9/train_data0', 'rb')
X_sequence_train, X_feature_train, X_image_train, Y_train = pickle.load(f1)
f1.close()
f2 = open('data/split_0.80_0.10_0.10_aug_to_2500_down_sample_True_instance0-9/val_data', 'rb')
X_sequence_val, X_feature_val, X_image_val, Y_val = pickle.load(f2)
f2.close()
f3 = open('data/split_0.80_0.10_0.10_aug_to_2500_down_sample_True_instance0-9/test_data', 'rb')
X_sequence_test, X_feature_test, X_image_test, Y_test = pickle.load(f3)
f3.close()

from keras.preprocessing.sequence import pad_sequences
X_sequence_train_paded = pad_sequences(X_sequence_train, dtype='float', padding='post', value = np.nan)
X_sequence_val_paded = pad_sequences(X_sequence_val, dtype='float', padding='post', value = np.nan)
X_sequence_test_paded = pad_sequences(X_sequence_test, dtype='float', padding='post', value = np.nan)
# 先pad再预处理，因为pad可以把array的list转为ndarray

from util import preprocess
X_sequence_train_trimed = preprocess(X_sequence_train_paded)
X_sequence_val_trimed = preprocess(X_sequence_val_paded)
X_sequence_test_trimed = preprocess(X_sequence_test_paded)

def build_channel(data):
    X = []
    for image in data:
        X.append(image.reshape(128, 128, 1))
    return X
X_image_train = np.array(build_channel(X_image_train))
X_image_val = np.array(build_channel(X_image_val))
X_image_test = np.array(build_channel(X_image_test))
X_feature_train = np.array(X_feature_train)
X_feature_val = np.array(X_feature_val)
X_feature_test = np.array(X_feature_test)
Y_train = np.array(Y_train)-1 # 文件夹名从1开始但是标签从0开始
Y_val = np.array(Y_val)-1
Y_test = np.array(Y_test)-1

In [ ]:
Y_train_one_hot = keras.utils.to_categorical(Y_train, num_classes=11)
Y_val_one_hot = keras.utils.to_categorical(Y_val, num_classes=11)
Y_test_one_hot = keras.utils.to_categorical(Y_test, num_classes=11)

In [ ]:
print(X_feature_test[0])

In [ ]:
f = open('data/split_10fold_No0_aug_to_15000_down_sample_False_instance0-9/class_weights1', 'rb')
train_weight, test_weight = pickle.load(f)
f.close()
print(train_weight)

In [ ]:
from mix_models import rnn_with_feature
from mix_models import cnn_with_feature1
from mix_models import cnn_with_feature2
from mix_models import multi_input_model


# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-3,
#     decay_steps=860,
#     decay_rate=0.85)

# boundaries = [2000]
    # values = [1e-3, 1e-4]
    # lr_schedule= tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    # boundaries, values, name=None
    # )

# strategy = tf.distribute.MirroredStrategy(['GPU:1'])
# print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
# with strategy.scope():


# model = multi_input_model(sequence_shape=(None, 3), image_shape=(128, 128, 1), 
#                         feature_shape=(2,), num_classes=11)
# model = rnn_with_feature(sequence_shape=(None, 3),feature_shape=(2,), num_classes=11)
model = cnn_with_feature1(image_shape=(128, 128, 1), feature_shape=(2,), num_classes=11)
# model = cnn(image_shape=(128, 128, 1), num_classes=11)
model.summary()
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    # loss="sparse_categorical_crossentropy",
    # loss=categorical_focal_loss(alpha=np.ones(11)),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    # metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    run_eagerly=False,
)
callbacks = [
  keras.callbacks.ModelCheckpoint(
    filepath='models/cnn_with_feature_test/model_{epoch}',
    save_freq='epoch'),
  keras.callbacks.TensorBoard(log_dir='models/cnn_with_feature_test/logs')
]

In [ ]:
# X_train = {'sequence':X_sequence_train_trimed, 'image':X_image_train, 'feature':X_feature_train}
# X_val = {'sequence':X_sequence_val_trimed, 'image':X_image_val, 'feature':X_feature_val}
# X_train = {'sequence':X_sequence_train_trimed, 'feature':X_feature_train}
# X_val = {'sequence':X_sequence_val_trimed, 'feature':X_feature_val}
X_train = {'image':X_image_train, 'feature':X_feature_train}
X_val = {'image':X_image_val, 'feature':X_feature_val}
model.fit(X_train, Y_train_one_hot, batch_size=32, epochs=50, 
        validation_data=(X_val, Y_val_one_hot), 
        #  class_weight=dict(enumerate(train_weight)),
        callbacks=callbacks)

In [ ]:
lodaed_model = keras.models.load_model('models/rnn_with_feature_test/model_6')
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from util import generate_batch

test_gen = generate_batch(X_sequence_test_trimed, X_image_test, X_feature_test, Y_test, 128)      
y_label = Y_test

y_predict = np.array([])
for x_sequence, x_image, x_feature, y in test_gen:
    y_predict = np.concatenate([y_predict, 
            np.argmax(lodaed_model.predict({'sequence':x_sequence, 
                                            # 'image':x_image,
                                            'feature':x_feature}), axis=-1)])
y_label = y_label.astype(np.int32) 
y_predict = y_predict.astype(np.int32) 
cm = confusion_matrix(y_true=y_label, y_pred=y_predict, normalize='true')

classes = ['RRab', 'RRc','RRd', 'Blazhko','Ecl', 'EA' , 'Rot', 'LPV','δ-Scuti','ACEP', 'Cep-II']
from util import cm_plot
plotting = cm_plot(cm, classes_types=classes, classifier_model='mix Classifier', nClasses=11,
                    cmap=plt.cm.Blues)